In [2]:
import numpy as np
from hmm_utils import HMM
from smoothing_state_attacker import ss_attacker
from extra import monte_carlo_enumeration
from collections import Counter
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [10]:
!pip install -user upgrade matplotlib


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -u


Package         Version
--------------- ---------
Bottleneck      1.3.2
certifi         2021.10.8
cycler          0.11.0
fonttools       4.25.0
hmmlearn        0.2.6
joblib          1.1.0
kiwisolver      1.3.1
matplotlib      3.5.1
mkl-fft         1.3.1
mkl-random      1.2.2
mkl-service     2.4.0
munkres         1.1.4
numexpr         2.8.1
numpy           1.21.0
olefile         0.46
packaging       21.3
pandas          1.2.5
Pillow          8.4.0
pip             21.2.4
pyparsing       3.0.4
python-dateutil 2.8.2
pytz            2021.1
scikit-learn    1.0.2
scipy           1.8.0
setuptools      58.0.4
sip             4.19.13
six             1.16.0
threadpoolctl   3.1.0
tornado         6.1
tqdm            4.62.3
wheel           0.37.1


In [4]:
#HMM A , B, pi

priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])
emission   = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                    [0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                       [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]])

In [5]:
# initialize HMM
hmm = HMM(n_components=3,n_obs=6)
hmm.startprob_ = priors
hmm.transmat_ = transition
hmm.emissionprob_ = emission

In [7]:
def linspace(start, stop, step=1.):
  """
    Like np.linspace but uses step instead of num
    This is inclusive to stop, so if start=1, stop=3, step=0.5
    Output is: array([1., 1.5, 2., 2.5, 3.])
  """
  return np.linspace(start, stop, int((stop - start) / step + 1))


In [8]:
x_vector = np.array([ [5], [3], [4], [3], [5]])
f1_attacker = ss_attacker(w1 = 1 ,w2= 0 ,t = 2, state= 0,c=1)
f2_attacker = ss_attacker(w1 = 0 ,w2= 1 ,t = 2, state= 0,c=1)
f1_utilities = monte_carlo_enumeration(hmm, 100, f1_attacker,  1, x_vector)[1]
f2_utilities = monte_carlo_enumeration(hmm, 100, f2_attacker,  1, x_vector)[1]
w1_range = linspace(0.1, 5, step= 0.1)
w2_range = linspace(0.1, 5, step= 0.1)
w1_w2_grid = np.array(np.meshgrid(w1_range, w2_range)).T.reshape(-1,2)

In [18]:
f2_utilities

array([-5., -5., -5., ..., -4., -4., -3.])

In [ ]:
3 * f1_utilities

In [12]:
Z_set = hmm.generate_z(len(x_vector))
ind_list = []
for comb in w1_w2_grid:
    w1, w2 = comb
    utility_vector = w1 * f1_utilities + w2 * f2_utilities
    max_indx = np.argmax(utility_vector)
    ind_list.append(max_indx)


In [1]:
## ratio plots
ratio_step = 0.1
final_ratio = 5
ss_att = ss_attacker(w1 = 1 ,w2= 0 ,t = 2, state= 0,c=1)
diff_list = []
ratio_list = []
for r in range(int(final_number/ratio)):
    utility_vector = w1 * f1_utilities + f2_utilities
    max_indx = np.argmax(utility_vector)
    z_star = Z_set[max_indx]
    y_vec =  hmm.attack_X(x_vector.reshape(5,1), np.ones([len(x_vector),6]),z_star).astype('int')
    compute_prob_att = ss_attacker(w1 = ss_att.w1 ,w2= ss_att.w2,t = ss_att.t, state=ss_att.state ,c=1)
    p_untainted_data = compute_prob_att.state_attraction_repulsion_f1(alpha = hmm.alpha(x_vector), beta = hmm.beta(x_vector))
    p_tainted_data = compute_prob_att.state_attraction_repulsion_f1(alpha = hmm.alpha(y_vec), beta = hmm.beta(y_vec))
    ratio_list.append(r)
    diff_list.append(abs(p_tainted_data - p_untainted_data))
    r+=0.1

NameError: name 'ss_attacker' is not defined

In [26]:
plt.plt(ratio_list, diff_list)

NameError: name 'plt' is not defined

In [ ]:
z_star = monte_carlo_enumeration(hmm, k_value, ss_att, N, x_vector)[0]

y_vec =  hmm.attack_X(x_vector.reshape(5,1), np.ones([len(x_vector),6]),z_star).astype('int')

compute_prob_att = ss_attacker(w1 = ss_att.w1 ,w2= ss_att.w2,t = ss_att.t, state=ss_att.state,c=1)
p_untainted_data = compute_prob_att.state_attraction_repulsion_f1(alpha = hmm.alpha(x_vector), beta = hmm.beta(x_vector))
p_tainted_data = compute_prob_att.state_attraction_repulsion_f1(alpha = hmm.alpha(y_vec), beta = hmm.beta(y_vec))

print('Best attack',  z_star)

print('..............................................................')

print('Probability of state ' , ss_att.state  ,' on untainted data: ', p_untainted_data )

print('...............................................................')

print('Probability of state ' , ss_att.state ,' on tainted data: ', p_tainted_data)

In [16]:
Z_set[7295]

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [15]:
Counter(ind_list)

Counter({7295: 2261, 3407: 216, 3405: 23})

In [56]:
w2_range

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. ])

In [33]:
 for i  in (np.array(np.meshgrid([0,1], [1,111,1,1,1,1,1,1,12,1,1])).T.reshape(-1,2)):
        print(i)

[0 1]
[  0 111]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[ 0 12]
[0 1]
[0 1]
[1 1]
[  1 111]
[1 1]
[1 1]
[1 1]
[1 1]
[1 1]
[1 1]
[ 1 12]
[1 1]
[1 1]


In [35]:
print(i)

[1 1]


In [36]:
a

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [46]:
linspace(0.1, 1, 0.1)

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [ ]:
w1 = linspace(start, stop, step=1.)
w2 = 